In [3]:
# --- 1. Módulos Principales de Ciencia de Datos y Finanzas ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st  # Importar sub-módulo stats es lo común en actuariales
import statsmodels.api as sm
import yfinance as yf

# --- 2. Manejo de Excel (Para outputs/inputs) ---
# Generalmente no se importan directamente a menos que necesites formato avanzado.
# Pandas usa openpyxl/xlsxwriter por detrás con 'to_excel'.
import xlsxwriter 
import openpyxl

# --- 3. Utilidades de Visualización e Interacción ---
import yaml
from IPython.display import display, Markdown # Útil para imprimir tablas bonitas o texto formateado en loops

# --- 4. Infraestructura (SOLO si estás manipulando archivos .ipynb programáticamente) ---
# NOTA CRÍTICA: Si solo vas a analizar datos, BORRA ESTAS LÍNEAS.
import nbformat
import nbconvert
import nbclient

Ejercicio Procesos estocásticos


In [ ]:
import numpy as np
from scipy.linalg import expm, inv
import sympy as sp

#Importamos numpy por que es necesario para trabajar con matrices
#Importamos expm para calcular la exponencial de matrices
#Importamos inv para calcular la inversa de matrices
#Importamos scipy.linalg para trabajar con funciones lineales algebraicas
#Importamos scipy para trabajar con funciones científicas
#Importamos sympy para trabajar con funciones simbólicas

# ==========================================
# MODELO DE CADENA DE MARKOV CONTINUA PARA ESTADOS DE CADENA DE PRODUCCIÓN.
# Aplicación: Máquina con 3 estados: Manual, Automático, Avería.
# Objetivo: Calcular probabilidades de estar en cada estado a lo largo del tiempo.

# Datos: Una cadena de producción sólo puede funcionar en modo automático, que es el
# habitual, o en modo manual, para realizar operaciones especiales o incrementar su ritmo de
# producción. El tiempo que pasa operando se distribuye de manera exponencial, con una media
# de 20 minutos en modo manual y de 30 minutos en modo automático. Por otra parte, si la
# cadena está en modo manual y cambia de estado, lo hace al modo automático con
# probabilidad de (2/3) o se avería. En cambio, si está operando en modo automático y se
# produce un cambio, lo hace al modo manual con probabilidad del 50% o se avería. Si la
# cadena se avería, el tiempo de reparación se distribuye de manera exponencial con media de 1
# hora y ha de arrancarse necesariamente en modo automático.

# Si este patrón de comportamiento se mantiene estable en el tiempo, y se considera
# como instante inicial el arranque de la cadena en modo automático.

#==========================================
#PASO 0: DEFINICIÓN DE PARÁMETROS INICIALES
#==========================================
print("0. DEFINICIÓN DE PARÁMETROS INICIALES ")
# Estados: 0=Manual, 1=Auto, 2=Avería
states = ['Manual', 'Auto', 'Avería']
n_states = len(states)

# Estado inicial: Arranque en modo automático
initial_state = np.array([0.0, 1.0, 0.0])  # Vector fila
print("Estado inicial (vector fila):", initial_state)

# Probabilidades de transicion entre estados
# Probabilidad de pasar de modo manual a automático: 2/3
p_manual_to_auto = 2/3
# Probabilidad de pasar de modo manual a avería: 1/3
p_manual_to_failure = 1-p_manual_to_auto
# Probabilidad de pasar de modo automático a manual: 1/2
p_auto_to_manual = 1/2
# Probabilidad de pasar de modo automático a avería: 1/2
p_auto_to_failure = 1-p_auto_to_manual
# Probabilidad de pasar de avería a automático: 1
p_failure_to_auto = 1.0

# Tiempos medios en cada estado (en horas)
mean_time_manual = 20 / 60  # 20 minutos en horas
mean_time_auto = 30 / 60    # 30 minutos en horas   
mean_time_failure = 1.0     # 1 hora
 
# El tiempo de permanencia inimterrumpida en cada estado se anota como \tau_i
# La media del tiempo de permanencia en cada estado es el inverso de la intensidad de paso q_i= -q_ii
# E(tau_i) = 1/q_i  => q_i = 1/E(tau_i)
q_m = 1 / mean_time_manual   # Intensidad de paso desde estado Manual
q_a = 1 / mean_time_auto     # Intensidad de paso desde estado Automático
q_av = 1 / mean_time_failure  # Intensidad de paso desde estado Avería

print("Intensidades de paso (q_i):")
print(" q_Manual =", q_m)
print(" q_Auto =", q_a)
print(" q_Avería =", q_av)

# Elementos de la diagonal principal en la matriz de intensidades de transición Q
q_mm = -q_m
q_aa = -q_a
q_avav = -q_av

print("Elementos de la diagonal principal en Q:")
print(" q_mm =", q_mm)
print(" q_aa =", q_aa)
print(" q_avav =", q_avav)

#Relacion principal entre intensidades de paso y probabilidades de transición:
# p*_ij = q_ij / q_i  para i != j
# p*_ii = 0
# donde p*_ij son las probabilidades de transición en la cadena integrada y discreta
# q_ij son las intensidades de transición en la cadena continua
# q_i es la intensidad de paso desde el estado i

q_ma = q_m * p_manual_to_auto      # Intensidad de transición de Manual a Auto
q_mav = q_m * p_manual_to_failure  # Intensidad de transición de Manual a Avería
q_am = q_a * p_auto_to_manual      # Intensidad de transición de Auto a Manual
q_aav = q_a * p_auto_to_failure    # Intensidad de transición de Auto a Avería
q_ava = q_av * p_failure_to_auto   # Intensidad de transición de Avería a Auto



# ==========================================
# PASO 1: Definición del Modelo (Matriz Q de Intensidades de transición. Elemento generador)
# ==========================================
print("1. DEFINICIÓN DE LA MATRIZ GENERADORA Q ")
# Intensidad de paso (q_i = -q_ii): Auto=2, Manual=3, Avería=1
# Filas: 0=Manual, 1=Auto, 2=Avería
# Intensidades de transición: elementos fuera de la diagonal
# Q es conservativa: las filas suman 0
Q = np.array([
    [-3.0,  2.0,  1.0],
    [ 1.0, -2.0,  1.0],
    [ 0.0,  1.0, -1.0]
])

print("Matriz Q:\n", Q)



# ==========================================
# PASO 2: Matriz de Saltos (Cadena Integrada  Y Discreta)
# ==========================================
print("\n--- 2. MATRIZ DE SALTOS (K) ---")
# K_ij = q_ij / -q_ii para i != j
# K_ii = 0

0. DEFINICIÓN DE PARÁMETROS INICIALES 
Estado inicial (vector fila): [0. 1. 0.]
Intensidades de paso (q_i):
 q_Manual = 3.0
 q_Auto = 2.0
 q_Avería = 1.0
1. DEFINICIÓN DE LA MATRIZ GENERADORA Q 
Matriz Q:
 [[-3.  2.  1.]
 [ 1. -2.  1.]
 [ 0.  1. -1.]]

--- 2. MATRIZ DE SALTOS (K) ---


In [ ]:
import numpy as np
from scipy.linalg import expm, inv

# ==========================================
# PASO 1: Definición del Modelo (Matriz Q de Intensidades de transición. Elemento generador)
# ==========================================
print("1. DEFINICIÓN DE LA MATRIZ GENERADORA Q ")
# Intensidad de paso (q_i = -q_ii): Auto=2, Manual=3, Avería=1
# Filas: 0=Manual, 1=Auto, 2=Avería
# Intensidades de transición: elementos fuera de la diagonal
# Q es conservativa: las filas suman 0
Q = np.array([
    [-3.0,  2.0,  1.0],
    [ 1.0, -2.0,  1.0],
    [ 0.0,  1.0, -1.0]
])

print("Matriz Q:\n", Q)

# ==========================================
# PASO 2: Matriz de Saltos (Cadena Integrada  Y Discreta)
# ==========================================
print("\n--- 2. MATRIZ DE SALTOS (K) ---")
# K_ij = q_ij / -q_ii para i != j
# K_ii = 0
diag_inv = np.diag(1 / -np.diag(Q)) # Matriz diagonal con 1/lambda_i
K = np.dot(diag_inv, Q) + np.eye(3) # Ajuste algebraico para obtener K
# Limpiamos errores de punto flotante en la diagonal
np.fill_diagonal(K, 0)

print("Matriz de Saltos K:\n", K)

# ==========================================
# PASO 3: Descomposición Espectral (Diagonalización)
# ==========================================
print("\n--- 3. DESCOMPOSICIÓN ESPECTRAL (Q = H * D * H_inv) ---")

# Calculamos autovalores y autovectores
eigenvalues, eigenvectors = np.linalg.eig(Q)

# Ordenamos para que coincidan con el informe (0, -2, -4)
# Nota: np.linalg.eig no garantiza orden, así que los identificamos
idx = eigenvalues.argsort()[::-1]   
lambdas = eigenvalues[idx]
H = eigenvectors[:, idx]

# Normalizamos H para que coincida con los vectores "bonitos" del informe (enteros)
# Multiplicamos columnas por constantes para limpiar la visualización
# (Esto no afecta al resultado final, es solo estético para igualar tu informe)
H[:,0] = H[:,0] / H[0,0]           # Lambda=0 -> vector de unos
H[:,1] = H[:,1] / H[2,1]           # Lambda=-2 -> vector (-1, -1, 1)
H[:,2] = H[:,2] / H[2,2]           # Lambda=-4 -> vector (-3, 5, 1)

print("Autovalores (Lambdas):", lambdas)
print("Matriz de Autovectores (H):\n", H)

# Calculamos la inversa de H
H_inv = inv(H)
print("Inversa de H (H_inv):\n", H_inv)

# Verificación de Distribución Estacionaria (Primera fila de H_inv)
pi_teorica = H_inv[0, :]
print("Distribución Estacionaria (pi):", pi_teorica)

# ==========================================
# PASO 4: Cálculo de P(t) = e^(Qt)
# ==========================================
print("\n--- 4. CÁLCULO DE PROBABILIDADES ---")

def calcular_Pt_exacta(t):
    # P(t) = H * exp(D*t) * H_inv
    D_t = np.diag(np.exp(lambdas * t))
    return H @ D_t @ H_inv

# a) Probabilidad t=12 (Medio día)
t_12 = 12
P_12 = calcular_Pt_exacta(t_12)
print(f"Matriz P(t={t_12}):\n", np.round(P_12, 4))
print(f"Probabilidad Auto -> Auto en t=12: {P_12[0,0]:.4f}")

# b) Probabilidad Conjunta: P(X_6=Man | X_0=Auto) * P(X_9=Av | X_6=Man)
t_step1 = 6
t_step2_duration = 3 # De t=6 a t=9 hay 3 horas

P_6 = calcular_Pt_exacta(t_step1)
P_3 = calcular_Pt_exacta(t_step2_duration)

prob_paso1 = P_6[0, 1] # Auto(0) -> Manual(1) en t=6
prob_paso2 = P_3[1, 2] # Manual(1) -> Avería(2) en delta_t=3

prob_conjunta = prob_paso1 * prob_paso2
print(f"\nCálculo Conjunto:")
print(f"P(Auto->Man, t=6): {prob_paso1:.6f}")
print(f"P(Man->Av, dt=3):  {prob_paso2:.6f}")
print(f"Probabilidad Final:  {prob_conjunta:.6f} ({prob_conjunta*100:.2f}%)")

--- 1. DEFINICIÓN DE LA MATRIZ GENERADORA Q ---
Matriz Q:
 [[-2.  1.  1.]
 [ 2. -3.  1.]
 [ 1.  0. -1.]]

--- 2. MATRIZ DE SALTOS (K) ---
Matriz de Saltos K:
 [[0.         0.5        0.5       ]
 [0.66666667 0.         0.33333333]
 [1.         0.         0.        ]]

--- 3. DESCOMPOSICIÓN ESPECTRAL (Q = H * D * H_inv) ---
Autovalores (Lambdas): [-5.55111512e-17 -2.00000000e+00 -4.00000000e+00]
Matriz de Autovectores (H):
 [[ 1. -1. -3.]
 [ 1. -1.  5.]
 [ 1.  1.  1.]]
Inversa de H (H_inv):
 [[ 3.75000000e-01  1.25000000e-01  5.00000000e-01]
 [-2.50000000e-01 -2.50000000e-01  5.00000000e-01]
 [-1.25000000e-01  1.25000000e-01  2.77555756e-17]]
Distribución Estacionaria (pi): [0.375 0.125 0.5  ]

--- 4. CÁLCULO DE PROBABILIDADES ---
Matriz P(t=12):
 [[0.375 0.125 0.5  ]
 [0.375 0.125 0.5  ]
 [0.375 0.125 0.5  ]]
Probabilidad Auto -> Auto en t=12: 0.3750

Cálculo Conjunto:
P(Auto->Man, t=6): 0.125002
P(Man->Av, dt=3):  0.498761
Probabilidad Final:  0.062346 (6.23%)
